In [1]:
import os
import torch
import torch.nn as nn
import timm
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

C:\Users\benas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class CTRDataset(Dataset):
    def __init__(self, folder_path, transform=None, ctr_min=0.0, ctr_max=0.27):
        self.folder_path = folder_path
        self.image_files = os.listdir(folder_path)
        self.transform = transform
        self.ctr_min = ctr_min
        self.ctr_max = ctr_max

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        filename = self.image_files[idx]
        ctr, _, _ = filename.split("_", 2)
        ctr = float(ctr) / 100.0 
        ctr = (ctr - self.ctr_min) / (self.ctr_max - self.ctr_min)
        img_path = os.path.join(self.folder_path, filename)
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor([ctr], dtype=torch.float32)

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


dataset = CTRDataset("filtered_images", transform)
val_size = int(0.2 * len(dataset))
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [ ]:


model = timm.create_model("deit_small_patch16_224", pretrained=True)

for param in model.parameters():
    param.requires_grad = False

for name, param in model.named_parameters():
    if name.startswith("blocks.10") or name.startswith("blocks.11") or name.startswith("norm"):
        param.requires_grad = True

with open("params.txt", "w") as f:
    for name, param in model.named_parameters():
        f.write(name + "\n")

in_features = model.head.in_features
model.head = nn.Sequential(
    nn.Linear(in_features, 256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(256, 1)
)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.head.parameters(), lr=1e-3)

best_val_loss = float('inf')
for epoch in range(10):
    model.train()
    running_loss = 0.0
    for imgs, targets in train_loader:
        imgs, targets = imgs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    model.eval()
    val_loss = 0.0
    all_preds, all_targets = [], []
    with torch.no_grad():
        for imgs, targets in val_loader:
            imgs, targets = imgs.to(device), targets.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, targets)
            val_loss += loss.item()

            all_preds.extend(outputs.cpu().numpy().flatten())
            all_targets.extend(targets.cpu().numpy().flatten())

    avg_train_loss = running_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    mae = mean_absolute_error(all_targets, all_preds)
    r2 = r2_score(all_targets, all_preds)

    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")


Epoch 1, Train Loss: 0.2253, Val Loss: 0.0979, MAE: 0.2300, R²: -5.0678
Epoch 2, Train Loss: 0.1211, Val Loss: 0.0506, MAE: 0.1713, R²: -2.3015
Epoch 3, Train Loss: 0.0554, Val Loss: 0.0284, MAE: 0.1318, R²: -0.9431
Epoch 4, Train Loss: 0.0405, Val Loss: 0.0219, MAE: 0.1134, R²: -0.4696
Epoch 5, Train Loss: 0.0294, Val Loss: 0.0230, MAE: 0.1139, R²: -0.5444
Epoch 6, Train Loss: 0.0223, Val Loss: 0.0162, MAE: 0.0912, R²: -0.0657
Epoch 7, Train Loss: 0.0184, Val Loss: 0.0155, MAE: 0.0861, R²: -0.0283
Epoch 8, Train Loss: 0.0174, Val Loss: 0.0155, MAE: 0.0890, R²: -0.0206
Epoch 9, Train Loss: 0.0156, Val Loss: 0.0129, MAE: 0.0768, R²: 0.1338
Epoch 10, Train Loss: 0.0106, Val Loss: 0.0124, MAE: 0.0743, R²: 0.1806
